In [1]:
# Import modules
from __future__ import print_function
import numpy as np
from matplotlib import pyplot as plt
import cv2
from augment_image_names import Generator
from data_loader import load_batch
from cnn_model import *
import pickle as pkl
import random
from cnn_model import *
import os
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import scipy
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops
from tensorflow.keras import backend as K

%matplotlib inline
np.random.seed(1)

import matplotlib.pyplot as plt

tf.test.gpu_device_name()

print(tf.__version__)

2.4.1


In [2]:
files = np.concatenate((np.linspace(10,200,20,dtype=int),np.linspace(250,1000,16,dtype=int),np.linspace(1000,83000,int((83000-1000)/100 + 1),dtype=int)))

base_dir = "/media/ros/de64b3cc-d3b4-44e1-b807-300d3d8adb21/ss_terrain_nav/data/"
tot_itr = 100
feed_test = np.array([['terrain5/run1/', '1592968304.1', '0.0', '0.0', '0.0', '0.0', '0.0']])
test_images, test_accels, test_sds, test_tf, test_ca = load_batch(feed_test,base_dir,train=False,down_scale=2)
test_images = np.expand_dims(test_images, axis=-1)

n = np.max(test_images)/5
print(n)

noise = np.random.normal(0,n,size=(tot_itr,test_images.shape[1],test_images.shape[2],test_images.shape[3]))
test_images = np.repeat(test_images, tot_itr, axis=0) + noise
test_ca = np.repeat(test_ca, tot_itr, axis=0)
test_tf = np.ones(test_images.shape)

saved_models = os.listdir(base_dir + "saved/")

0.1780392156862745


pics = []
files = os.listdir(base_dir)
for f in files:
    if f.endswith("uncertainty.png"):\
        pics.append(f[:-15])

saved_model = np.setdiff1d(pics,saved_models)

In [ ]:
for m in saved_models:
    model = tf.keras.models.load_model(base_dir + "saved/" + m)
    preds = model.predict([test_images,test_ca,test_tf])
    z_sd = test_sds[:,:,:,2]
    pred_sd = np.std(preds,axis=0)
    xs = z_sd.flatten()
    ys = pred_sd.flatten()
    zers = np.argwhere(xs==-1)
    xs[zers] = 0
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(xs,ys)
    ax.set_xlabel("Uncertainty in Data")
    ax.set_ylabel("Uncertainty in Prediction")
    fig.savefig(base_dir + m + "_{}_uncertainty.png".format(n))


    fig = plt.figure()
    z_pred_sd = np.reshape(pred_sd,(pred_sd.shape[0],pred_sd.shape[1]))
    plt.imshow(z_pred_sd, cmap='hot', interpolation='nearest')
    fig.savefig(base_dir + m + "_{}_heatmap.png".format(n))
    print(m)
    plt.close('all')
    
    K.clear_session()

/home/ros/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


40-2550.65


In [ ]:
os.system('spd-say "Done with training"')
duration = 1  # seconds
freq = 440  # Hz
os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

###### 